In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service
import pandas as pd
import json
from selenium.webdriver.common.by import By
from time import sleep
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def init_driver():
    options = ChromeOptions()
    #options.add_argument("--headless=new")
    options.set_capability("goog:loggingPrefs", {"performance": "ALL"})
    options.add_experimental_option("perfLoggingPrefs", {"enableNetwork": True})
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def find_element_wait(driver, selector):
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    element = driver.find_element(By.CSS_SELECTOR, selector)
    return element

def find_elements_wait(driver, selector):
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector)))
    elements = driver.find_elements(By.CSS_SELECTOR, selector)
    return elements

def collect_links(Dimension, data_links):
    df = pd.DataFrame(columns=[Dimension, 'Link'])
    for data_link in data_links:
        print(f"{data_link.text}")
        print(f"Navigating to {data_link.get_attribute('href')}...")
        link_dict = {Dimension: data_link.text, 'Link': data_link.get_attribute('href')}
        df = pd.concat([df, pd.DataFrame([link_dict])], ignore_index=True)        
    return df

def collect_dimension_links(year_df):
    year_df_2 = pd.DataFrame(columns=['Year', 'Year_Link', 'Dimension', 'Dimension_Link'])
    for index, row in year_df.iterrows():
        year = row['Year']
        year_link = row['Link']
        web_driver = init_driver()
        print(f"Navigating to {year_link}...")
        web_driver.get(year_link) 
        dimension_links = find_elements_wait(driver=web_driver, selector='div[class="ListR"] > a')     
        for data_link in dimension_links:
            dimension = data_link.text
            dimension_link = data_link.get_attribute("href")
            print(dimension)
            print(dimension_link)            
            link_dict = {'Year': year, 'Year_Link': year_link, 'Dimension': dimension, 'Dimension_Link': dimension_link}      
            year_df_2 = pd.concat([year_df_2, pd.DataFrame([link_dict])], ignore_index=True)

    return year_df_2
        

#WebDriverWait(web_driver, 30).until( EC.presence_of_element(By.CSS_SELECTOR, 'ul > li[class="class_A"]')_located((By.CSS_SELECTOR, 'ul > li[class="class_A"] > a')))



In [ ]:
web_driver = init_driver()
data_index_page = "http://www.pbc.gov.cn/en/3688247/3688975/index.html"
web_driver.get(data_index_page)
year_links = find_elements_wait(driver=web_driver, selector='ul > li[class="class_A"] > a')
year_df = collect_links('Year', year_links)
year_df_sample=year_df.head(3)
print(year_df_sample)
year_df_2=collect_dimension_links(year_df_sample)
year_df_3 = pd.DataFrame(columns=['Year', 'Year_Link', 'Dimension', 'Dimension_Link', 'Data', 'Data_Link'])

for index, row in year_df_2.iterrows():
        print(row)
        year = row['Year']
        year_link = row['Year_Link']
        dimension=row['Dimension']
        dimension_link=row['Dimension_Link']
        web_driver = init_driver()
        web_driver.get(dimension_link)   
        table_rows = find_elements_wait(driver=web_driver, selector='table[class="data_table"] tbody > tr')
        table_cells = find_elements_wait(driver=web_driver, selector='table[class="data_table"] tbody > tr > td')
        for cell in table_cells:
            if cell.get_attribute("class") != "data_xz":
                row_name = cell.text
                print(row_name)
            elif cell.get_attribute("class") == "data_xz" and cell.find_element(By.CSS_SELECTOR, 'a').text == "xls":
                #print(cell.find_element(By.CSS_SELECTOR, 'a'))
                xls_link = cell.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                print(xls_link)
            link_dict = {'Year': year, 'Year_Link': year_link, 'Dimension': dimension, 'Dimension_Link': dimension_link,'Data':row_name,'Data_Link':xls_link}      
            year_df_3 = pd.concat([year_df_3, pd.DataFrame([link_dict])], ignore_index=True)
        web_driver.quit()

In [ ]:
year_df_sample=year_df.head(3)
print(year_df_sample)

In [ ]:
year_df_2=collect_dimension_links(year_df_sample)

In [ ]:
year_df_2.to_csv('year_df_2.csv', index=False)

In [ ]:
year_df_3 = pd.DataFrame(columns=['Year', 'Year_Link', 'Dimension', 'Dimension_Link', 'Data', 'Data_Link'])
for index, row in year_df_2.iterrows():
        print(row)
        year = row['Year']
        year_link = row['Year_Link']
        dimension=row['Dimension']
        dimension_link=row['Dimension_Link']
        web_driver = init_driver()
        web_driver.get(dimension_link)   
        table_rows = find_elements_wait(driver=web_driver, selector='table[class="data_table"] tbody > tr')
        table_cells = find_elements_wait(driver=web_driver, selector='table[class="data_table"] tbody > tr > td')
        for cell in table_cells:
            if cell.get_attribute("class") != "data_xz":
                row_name = cell.text
                print(row_name)
            elif cell.get_attribute("class") == "data_xz" and cell.find_element(By.CSS_SELECTOR, 'a').text == "xls":
                #print(cell.find_element(By.CSS_SELECTOR, 'a'))
                xls_link = cell.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
                print(xls_link)
            link_dict = {'Year': year, 'Year_Link': year_link, 'Dimension': dimension, 'Dimension_Link': dimension_link,'Data':row_name,'Data_Link':xls_link}      
            year_df_3 = pd.concat([year_df_3, pd.DataFrame([link_dict])], ignore_index=True)
        web_driver.quit()

In [ ]:
print(year_df_3)

In [ ]:
year_df_3.to_csv('year_df_3.csv', index=False)